In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('whatsapp.txt',sep='\t',header=None)

In [3]:
data.drop(0,inplace=True)

In [4]:
message=data[0].str.split(',',n=3,expand=True)

In [5]:
data[['Date','Time','Name','Message']]=message

In [6]:
data.drop(0,axis=1,inplace=True)

In [7]:
def label_flirt(row):
    flirt_keywords = ['love ','😉','😊','❤️','care','fun','romantic','tastes','outing']
    for keyword in flirt_keywords:
        if keyword in row['Message'].lower():
            return 'Flirt'
    return 'Not Flirt'

In [8]:
data['Labels']=data.apply(label_flirt,axis=1)

In [74]:
data

,Date,Time,Name,Message,Labels
0,2023-08-05 00:00:00,10:00 AM,Arun,Hey Hema!,Not Flirt
1,2023-08-05 00:00:00,10:05 AM,Hema,Hi Arun! How's your day going?,Not Flirt
2,2023-08-05 00:00:00,10:10 AM,Arun,"It's going well, just thinking about you 😉",Flirt
3,2023-08-05 00:00:00,10:15 AM,Hema,"Haha, smooth! I'm flattered 😊",Flirt
4,2023-08-05 00:00:00,10:20 AM,Arun,You always make my day better!,Not Flirt
...,...,...,...,...,...
68,08/05/2023,10:45 AM,Hema,Looking forward to seeing you!,Not Flirt
69,08/05/2023,10:50 AM,Arun,Let's meet up soon 😊,Flirt
70,08/05/2023,10:55 AM,Hema,<Media omitted>,Not Flirt
71,08/05/2023,11:00 AM,Arun,Can we talk later?,Not Flirt


In [71]:
new_messages = pd.DataFrame({
    'Date': ['08/05/2023', '08/05/2023', '08/05/2023', '08/05/2023', '08/05/2023', '08/05/2023', '08/05/2023', '08/05/2023', '08/05/2023', '08/05/2023'],
    'Time': ['10:20 AM', '10:25 AM', '10:30 AM', '10:35 AM', '10:40 AM', '10:45 AM', '10:50 AM', '10:55 AM', '11:00 AM', '11:05 AM'],
    'Name': ['Arun', 'Hema', 'Arun', 'Hema', 'Arun', 'Hema', 'Arun', 'Hema', 'Arun', 'Hema'],
    'Message': [
        'You always make my day better!', 
        '<Media omitted>', 
        'Did you see the news today?', 
        'Missed call from Arun',
        'Are you coming to the party?', 
        'Looking forward to seeing you!',
        'Let\'s meet up soon 😊', 
        '<Media omitted>', 
        'Can we talk later?', 
        'Missed call from Hema'
    ],
    'Labels': ['Not Flirt', 'Not Flirt', 'Not Flirt', 'Not Flirt', 'Not Flirt', 'Not Flirt', 'Flirt', 'Not Flirt', 'Not Flirt', 'Not Flirt']
})

# Append new messages to existing data
data = pd.concat([data, new_messages], ignore_index=True)

data

,Date,Time,Name,Message,Labels,Predicted_Labels
0,2023-08-05 00:00:00,10:00 AM,Arun,Hey Hema!,Not Flirt,Not Flirt
1,2023-08-05 00:00:00,10:05 AM,Hema,Hi Arun! How's your day going?,Not Flirt,Not Flirt
2,2023-08-05 00:00:00,10:10 AM,Arun,"It's going well, just thinking about you 😉",Flirt,Flirt
3,2023-08-05 00:00:00,10:15 AM,Hema,"Haha, smooth! I'm flattered 😊",Flirt,Flirt
4,2023-08-05 00:00:00,10:20 AM,Arun,You always make my day better!,Not Flirt,Not Flirt
...,...,...,...,...,...,...
68,08/05/2023,10:45 AM,Hema,Looking forward to seeing you!,Not Flirt,NaN
69,08/05/2023,10:50 AM,Arun,Let's meet up soon 😊,Flirt,NaN
70,08/05/2023,10:55 AM,Hema,<Media omitted>,Not Flirt,NaN
71,08/05/2023,11:00 AM,Arun,Can we talk later?,Not Flirt,NaN


In [73]:
data.drop('Predicted_Labels',axis=1,inplace=True)

In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression,PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

In [76]:
# Define vectorizers
count_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()

In [77]:
X=data['Message']
y=data['Labels']

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [79]:
# Vectorize the training data
X_train_cv = count_vectorizer.fit_transform(X_train)
X_test_cv = count_vectorizer.transform(X_test)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [80]:
# Define classifiers
nb_classifier = MultinomialNB()
lr_classifier = LogisticRegression()
pa_classifier=PassiveAggressiveClassifier(max_iter=50)

In [81]:
# Train and evaluate models
def train_and_evaluate(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, zero_division=0))
    return model

In [82]:
# Count Vectorizer with Naive Bayes
print("Count Vectorizer with Naive Bayes")
cv_nb_model = train_and_evaluate(nb_classifier, X_train_cv, X_test_cv, y_train, y_test)

# TF-IDF Vectorizer with Naive Bayes
print("TF-IDF Vectorizer with Naive Bayes")
tfidf_nb_model = train_and_evaluate(nb_classifier, X_train_tfidf, X_test_tfidf, y_train, y_test)

# Count Vectorizer with Logistic Regression
print("Count Vectorizer with Logistic Regression")
cv_lr_model = train_and_evaluate(lr_classifier, X_train_cv, X_test_cv, y_train, y_test)

# TF-IDF Vectorizer with Logistic Regression
print("TF-IDF Vectorizer with Logistic Regression")
tfidf_lr_model = train_and_evaluate(lr_classifier, X_train_tfidf, X_test_tfidf, y_train, y_test)

# Count Vectorizer with Passive agression
print("Count Vectorizer with Passive agression")
cv_ar_model = train_and_evaluate(pa_classifier, X_train_cv, X_test_cv, y_train, y_test)

# TF-IDF Vectorizer with Passive agression
print("TF-IDF Vectorizer with Passive agression")
tfidf_ar_model = train_and_evaluate(pa_classifier, X_train_tfidf, X_test_tfidf, y_train, y_test)

Count Vectorizer with Naive Bayes
              precision    recall  f1-score   support

       Flirt       0.50      0.33      0.40         3
   Not Flirt       0.85      0.92      0.88        12

    accuracy                           0.80        15
   macro avg       0.67      0.62      0.64        15
weighted avg       0.78      0.80      0.78        15

TF-IDF Vectorizer with Naive Bayes
              precision    recall  f1-score   support

       Flirt       0.50      0.33      0.40         3
   Not Flirt       0.85      0.92      0.88        12

    accuracy                           0.80        15
   macro avg       0.67      0.62      0.64        15
weighted avg       0.78      0.80      0.78        15

Count Vectorizer with Logistic Regression
              precision    recall  f1-score   support

       Flirt       1.00      0.33      0.50         3
   Not Flirt       0.86      1.00      0.92        12

    accuracy                           0.87        15
   macro avg     

In [83]:
# Choose the best model based on the report 
best_model = tfidf_nb_model
X_vectorized = tfidf_vectorizer.transform(data['Message'])
data['Predicted_Labels'] = best_model.predict(X_vectorized)

In [89]:
data['Date']=pd.to_datetime(data['Date'])

In [90]:
data

,Date,Time,Name,Message,Labels,Predicted_Labels
0,2023-08-05,10:00 AM,Arun,Hey Hema!,Not Flirt,Not Flirt
1,2023-08-05,10:05 AM,Hema,Hi Arun! How's your day going?,Not Flirt,Not Flirt
2,2023-08-05,10:10 AM,Arun,"It's going well, just thinking about you 😉",Flirt,Not Flirt
3,2023-08-05,10:15 AM,Hema,"Haha, smooth! I'm flattered 😊",Flirt,Flirt
4,2023-08-05,10:20 AM,Arun,You always make my day better!,Not Flirt,Not Flirt
...,...,...,...,...,...,...
68,2023-08-05,10:45 AM,Hema,Looking forward to seeing you!,Not Flirt,Not Flirt
69,2023-08-05,10:50 AM,Arun,Let's meet up soon 😊,Flirt,Flirt
70,2023-08-05,10:55 AM,Hema,<Media omitted>,Not Flirt,Not Flirt
71,2023-08-05,11:00 AM,Arun,Can we talk later?,Not Flirt,Not Flirt


# Additional Analyses


In [91]:
# Talkative and Less Talkative
talk=data['Name'].value_counts()



In [92]:
More_talkative=talk.idxmax()
Less_talkative=talk.idxmin()
print(f' More Talkative :{More_talkative}')
print(f' Less Talkative :{Less_talkative}')


 More Talkative :Arun
 Less Talkative :Shweta


In [96]:
Most_Active_Date=data['Date'].dt.date.value_counts().idxmax()
print(f' Most_Active_Date :{Most_Active_Date}')


 Most_Active_Date :2023-08-05


In [99]:
Most_Active_Time=data['Time'].value_counts().idxmax()
print(f' Most_Active_Time :{Most_Active_Time}')


 Most_Active_Time :10:40 AM


In [123]:
media= data[data['Message'].str.contains('<Media omitted>')]['Name'].value_counts().reset_index()
print(f'Media count sent by each person: {media['count'][0]}')

Media count sent by each person: 6


In [126]:
missed_call= data[data['Message'].str.contains('Missed')]['Name'].value_counts().reset_index()
print(f'Media count sent by each person: {missed_call['count'][0]}')

Media count sent by each person: 6


In [127]:
data

,Date,Time,Name,Message,Labels,Predicted_Labels
0,2023-08-05,10:00 AM,Arun,Hey Hema!,Not Flirt,Not Flirt
1,2023-08-05,10:05 AM,Hema,Hi Arun! How's your day going?,Not Flirt,Not Flirt
2,2023-08-05,10:10 AM,Arun,"It's going well, just thinking about you 😉",Flirt,Not Flirt
3,2023-08-05,10:15 AM,Hema,"Haha, smooth! I'm flattered 😊",Flirt,Flirt
4,2023-08-05,10:20 AM,Arun,You always make my day better!,Not Flirt,Not Flirt
...,...,...,...,...,...,...
68,2023-08-05,10:45 AM,Hema,Looking forward to seeing you!,Not Flirt,Not Flirt
69,2023-08-05,10:50 AM,Arun,Let's meet up soon 😊,Flirt,Flirt
70,2023-08-05,10:55 AM,Hema,<Media omitted>,Not Flirt,Not Flirt
71,2023-08-05,11:00 AM,Arun,Can we talk later?,Not Flirt,Not Flirt


## Save the model using pickle

In [128]:
#Save the model using pickle
import pickle
filename='Finalised_whatsapp.sav'
pickle.dump(tfidf_nb_model,open(filename,'wb'))

In [129]:
load_model=pickle.load(open(filename,'rb'))